In [ ]:
import pymc3 as pm 
import numpy as np 
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd 
import os 

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 13}

plt.rc('font', **font)
plt.style.use('classic')

# Data
cwd = os.chdir("/Users/danbarfod/Downloads/Kula_Biotite")

In [ ]:
df6 = pd.read_csv("Kula_6_data.csv", engine = 'python')
df6.dropna(axis =0, how = 'all')

temp1 = df6['Temp'].values
time1 = df6['Time'].values
a391 = df6['39Ar'].values
a401 = df6['40Ar'].values
age1 = df6['Age'].values
age1_err = df6['Age_Err'].values

In [ ]:
# Shared variable for modelling 
from theano import shared, tensor as tt
data = age1
err = age1_err

age_err = shared(err*3)

In [ ]:
def Increment(ea, d0, a, temp, t):
    R = 8.314e-3
    
    D0t = (d0/(a**2))*np.exp(-ea/(R*temp))*t
    
    q = (6./(np.pi**(3/2)))*np.sqrt((np.pi**2)*D0t)
    
    aa1 = np.sqrt(np.pi) * R * temp**2
    bb1 = d0*np.exp(-ea/(R*temp))*t
    z1 = (3*ea *np.sqrt(bb1/a**2))/aa1
    

    aa2 = np.sqrt(np.pi) * R *temp**2
    bb2 = d0*np.exp(-ea/(R*temp))*t
    z2 = (3*ea*(np.sqrt(bb2/a**2) - np.sqrt(np.pi)*bb2/a**2))/aa2
      
    
    aa3 = a**2 * R * temp**2
    bb3 = d0*np.exp(-ea/(R*temp))*t
    z3 = 6 * d0 * ea * t * np.exp((-np.pi**2 * bb3/a**2) - ea/(R*temp))/aa3

    
    Q = tt.switch(q <= 0.9,
                z2,
                 z3)
    
    T = tt.switch(q < 0.1, z1, Q)
    
    
    return T

In [ ]:
def fracrelx(Ea, D0, a, temp, time): 
    R = 8.314e-3
    D0t = (D0/(a**2))*np.exp(-Ea/(R*temp))*time
    
    q = (6./(np.pi**(3/2)))*np.sqrt((np.pi**2)*D0t)
    
    q1 = (6./(np.pi**(1.5)))*np.sqrt((np.pi**2)*D0t)
    
    q2 = (6./(np.pi**(3/2)))*np.sqrt((np.pi**2)*D0t)\
    - (3./(np.pi**2)) *((np.pi**2)* D0t)
    
    q3 = 1 - (6./(np.pi**2)) *np.exp(-(np.pi**2)*D0t)
     
    
    
    Q = tt.switch(q <= 0.9, q2, q3)
    T = tt.switch(q < 0.1, q, Q)
    
    return T

In [ ]:
def frel_data(x): 
    y = np.cumsum(x) 
    dd = y/y.max()
    return dd

In [ ]:
x = 100 *frel_data(a391)

# The Age Spectra

In [ ]:
y = np.insert(x, 0, 0)
age_plot = np.insert(data, 0, data[0])
age_plot_err = np.insert(err, 0, err[0])


fig, ax = plt.subplots(1,2, figsize = (9, 4.5))

ax[0].fill_between(y, age_plot - age_plot_err, age_plot + age_plot_err, 
                  step = 'pre', 
                  color = 'grey', alpha = 0.7, 
                  edgecolor = 'k')
ax[0].set_xlim(y.min(), y.max())

ax[0].set_ylabel('Age (Ma)')
ax[0].set_xlabel('Cumulative $^{39}$Ar released (%)')

ax[1].plot(temp1,x, lw = 2, 
          color= 'k')
ax[1].set_ylabel('Cumulative $^{39}$Ar released (%)')
ax[1].set_xlabel('Temperature ($^{o}C$)')
ax[1].set_ylim(0, 101)
plt.tight_layout()

In [ ]:
data_f = x/100
tempk = temp1 + 273.15
t_obs = time1 * 60
N = data.shape[0]
R = 8.314e-3

In [ ]:
Q = pm.Model()

with Q: 
    
    phi = pm.Dirichlet('phi', 
                      a = np.array([0.25,0.75]), 
                      shape = 2, 
                       testval = (0.25, 0.75))
    
    w1 = phi[0]
    w2 = phi[1]
    
    w1_potential = pm.Potential('p1_con', 
                               tt.switch(w1 < 0.2, 
                                        -np.inf, 0) + 
                               tt.switch(w1 > 0.3, 
                                        -np.inf, 0))
    
    ea1 = pm.Normal('ea1', mu = 196,
                    sd = 2, 
                   testval = 196)
    ea2 = pm.Normal('ea2', mu = 196,
                    sd = 2, 
                   testval = 196)
    
    d01 = pm.TruncatedNormal('d01', 
                            mu = 0.002, sd = 0.2, 
                            lower = 0.0)
    
    d02 = pm.TruncatedNormal('d02', 
                            mu = 0.002, sd = 0.2, 
                            lower = 0.0)
    
    a1 = pm.Uniform('a1', lower = 0.0177/2, 
                   upper= 0.025/2)
    
    a2 = pm.Uniform('a2', lower = 0.0177/2, 
                   upper= 0.025/2)

In [ ]:
with Q: 
    
    age1 = pm.Uniform('age1', lower = 0.0, upper = 4600, 
                     testval = 200)
    
    age2 = pm.Uniform('age2', lower = 0.0, upper = 4600, 
                     testval = 100)
    
    age_cons = pm.Potential('age_con', 
                           tt.switch(age1 - age2 < 0, 
                                    -np.inf, 0))

In [ ]:
with Q: 

    X = Increment(ea1, d01, a1, tempk, t_obs)
    Y = Increment(ea2, d02, a2, tempk, t_obs)

    
    mu2 = ((w1*X)/(w1*X + w2*Y))*age1 +\
    ((w2*Y)/(w1*X + w2*Y))*age2   

    model2 = pm.Deterministic('model2', mu2)

In [ ]:
with Q: 
    # Cumulative Release Model 

    A = fracrelx(ea1, d01, a1, tempk, t_obs)
    B = fracrelx(ea2, d02, a2, tempk, t_obs)
    
    # Individual Cumulative Release
    AA = pm.Deterministic('AA', A)
    BB = pm.Deterministic('BB', B)
    
    mu1 = w1*A + w2*B

    # MONOTONIC Constraint
######################################################################
    for i in range(len(tempk) -1): 
        F_con1 = pm.Potential(f'F_con1{i}', 
                             tt.switch(A[i + 1] - A[i] <0, 
                                      -np.inf, 0) + 
                             tt.switch(B[i +1] - B[i] < 0, 
                                      -np.inf, 0))

    for i in range(len(tempk) -1): 
        F_con2 = pm.Potential(f'F_con2{i}', 
                             tt.switch(mu1[i + 1] - mu1[i] <0, 
                                      -np.inf, 0))
        
    # Constraint to the fractional release model 
    frac_rel_cons = pm.Potential('frac_rel_cons', 
                                tt.switch(mu1 >1, -np.inf, 0))
######################################################################    

    
    # Fractional Release Equation 
    f_err1 = pm.HalfNormal('f_err1', 
                           sd = 0.1, 
                          shape = N)

In [ ]:
# Models for plotting 
with Q: 
    f_model = pm.Deterministic('f_model', 
                               mu1)
    age_model = pm.Deterministic('Age_Model', 
                                mu2)

In [ ]:
def logp(mod, data, err): 
    logp = -0.5*tt.sum((((data - mod)/err)**2))
    return logp

In [ ]:
with Q: 
    age_model_error = pm.HalfNormal('age_model_error', 
                                   sd = 5, 
                                   shape = N)

In [ ]:
with Q: 
    # Cumulative Release Likelihood 
    L1 = pm.Normal('L1', mu = mu1, 
                   sd = f_err1, 
                   observed = data_f)
    
    L2 = pm.DensityDist('L2', logp, 
                       observed = {"mod":mu2, 
                                  "data":data, 
                                  "err":age_model_error})

In [ ]:
SAMPLES = 120000
BURN = 100000
with Q: 
    
    step1 = pm.NUTS([phi], target_accept = 0.93)
    
    step2 = pm.Metropolis([ea1, 
                           ea2, 
                           age1, age2,
                           a1, a2,
                           f_err1, 
                          d01, d02])

    
    trace = pm.sample(SAMPLES, tune = BURN, 
                     step = [step1, step2])

In [ ]:
x1 = trace['f_model'][-5000:].mean(axis = 0)
plt.figure(figsize = (5,5))
plt.plot(temp1, x1)
plt.plot(temp1, data_f)

In [ ]:
x2 = trace['Age_Model'][-10000:].mean(axis = 0)

x1_plot = np.insert(x1, 0, 0)
x2_plot = np.insert(x2, 0, x2[0])
plt.figure(figsize = (5,5))
plt.plot(100*x1_plot, x2_plot)
plt.plot(y, age_plot)

In [ ]:
# 95% Credible model intervals 
low_age, high_age = np.percentile(trace['Age_Model'][-SAMPLES::2],
                                  [2.5, 97.5], 
                                  axis=0)
low_f, high_f = np.percentile(trace['f_model'][-SAMPLES::2], 
                             [2.5, 97.5], 
                             axis = 0)

In [ ]:
low_age_plot = np.insert(low_age, 0, low_age[0])
high_age_plot = np.insert(high_age, 0, high_age[0])

In [ ]:
y = np.insert(x, 0, 0)
age_plot = np.insert(data, 0, data[0])
age_plot_err = np.insert(err, 0, err[0])


fig, ax = plt.subplots(1,2, figsize = (9, 4.5))
# Age
ax[0].fill_between(y, age_plot - age_plot_err, age_plot + age_plot_err, 
                  step = 'pre', 
                  color = 'grey', alpha = 0.7, 
                  edgecolor = 'k')
ax[0].set_xlim(y.min(), y.max())


ax[0].plot(100*x1_plot, x2_plot, 'r-')
ax[0].fill_between(100*x1_plot, low_age_plot, 
                   high_age_plot,
                  step = 'pre', 
                  color = 'r', alpha = 0.7, 
                  edgecolor = 'k')

ax[0].set_ylabel('Age (Ma)')
ax[0].set_xlabel('Cumulative $^{39}$Ar released (%)')

ax[0].set_ylim(65, 125)


# Cumulative Release

ax[1].plot(temp1,x/100, lw = 2, 
          color= 'k')
ax[1].plot(temp1,x1, lw = 2, 
          color= 'r')
ax[1].fill_between(temp1, low_f, 
                   high_f,
                  color = 'r', alpha = 0.7, 
                  edgecolor = 'k')
ax[1].set_ylabel('Cumulative $^{39}$Ar released (%)')
ax[1].set_xlabel('Temperature ($^{o}C$)')
ax[1].set_ylim(0, 1.01)
plt.tight_layout()

In [ ]:
print(trace['age1'][-5000::2].mean(axis =0))
print(trace['age2'][-5000::2].mean(axis =0))


print(2*np.std(trace['age1'][-5000::2]))
print(2*np.std(trace['age2'][-5000::2]))

#### pm.summary(trace)

# Posterior DataFrame

In [ ]:
df = pd.DataFrame({"age_model": x2,
                   "age_model_low":low_age, 
                   "age_model_high":high_age, 
                   "frel_model": x1, 
                   "frel_model_low": low_f, 
                   "frel_model_high":high_f, 
                   "age_data": data, 
                   "age_err": err,
                   "frel_data":x/100, 
                   "temp":temp1
                  })

In [ ]:
df.to_csv("Kula_biotite_3_posterior.csv")

In [ ]:
df2 = pm.trace_to_dataframe(trace[-SAMPLES::5])
df2.to_csv("Kula_biotite_3_trace.csv")